## Fake NEws Classificer USsing Bidirectional LSTM

In [1]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df=pd.read_csv('/content/drive/MyDrive/train (1).csv')

In [ ]:
#df=pd.read_csv('train.csv',engine='python',error_bad_lines=False)

Skipping line 7100: unexpected end of data


In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df.shape

(20800, 5)

In [7]:
df=df.dropna()

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
## get the independent features
X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
## Check whether dataset is balanced or not
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.14.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [15]:
## vocabulary size
voc_size=5000

In [16]:
messages=X.copy()

In [17]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
messages.reset_index(inplace=True)

In [23]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [25]:
#corpus

In [26]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2410, 1586, 3277, 1425, 4239, 2159, 2909, 3310, 1414, 2817],
 [4159, 2224, 2319, 4556, 3827, 2943, 3525],
 [2563, 2883, 3272, 1654],
 [3454, 4701, 1298, 3127, 2671, 584],
 [2052, 3827, 4614, 2221, 2645, 3136, 3827, 4574, 1965, 788],
 [2866,
  2396,
  1229,
  1811,
  1202,
  1469,
  4787,
  3099,
  2235,
  2406,
  4902,
  3587,
  830,
  3430,
  3525],
 [1496, 49, 3886, 3398, 2582, 885, 2422, 3885, 2958, 3717, 1240],
 [3385, 1084, 3630, 68, 4269, 3757, 1469, 3828, 2958, 3717, 1240],
 [3066, 4600, 94, 1846, 4161, 3029, 786, 4461, 1469, 3659],
 [3279, 3889, 4261, 584, 1631, 2628, 2213, 3771],
 [3799, 4861, 4830, 3045, 325, 2591, 2633, 2803, 2251, 3649, 1420],
 [3127, 761, 4239, 3029, 1469, 4269],
 [3543, 439, 1137, 4912, 1386, 4128, 3193, 3964, 210],
 [801, 1995, 1751, 3341, 4202, 961, 3763, 2958, 3717, 1240],
 [1235, 4778, 4644, 2013, 4360, 2958, 3717, 1240],
 [4940, 2434, 3691, 4962, 3115, 4654, 1515, 817, 890, 63],
 [1774, 744, 2224],
 [3694, 1356, 4580, 4303, 1469, 3749, 4050, 3525],

## Embedding Representation

In [27]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 3310, 1414, 2817],
       [   0,    0,    0, ..., 3827, 2943, 3525],
       [   0,    0,    0, ..., 2883, 3272, 1654],
       ...,
       [   0,    0,    0, ..., 2958, 3717, 1240],
       [   0,    0,    0, ...,   90, 1479, 4806],
       [   0,    0,    0, ..., 3672, 3992, 3847]], dtype=int32)

In [28]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [36]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=32)

Epoch 1/5
383/383 [==============================] - 5s 14ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 0.6871 - val_accuracy: 0.9097
Epoch 2/5
383/383 [==============================] - 3s 8ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.7101 - val_accuracy: 0.9082
Epoch 3/5
383/383 [==============================] - 3s 8ms/step - loss: 7.8116e-04 - accuracy: 0.9999 - val_loss: 0.6972 - val_accuracy: 0.9099
Epoch 4/5
383/383 [==============================] - 4s 10ms/step - loss: 0.0020 - accuracy: 0.9993 - val_loss: 0.6459 - val_accuracy: 0.9100
Epoch 5/5
383/383 [==============================] - 5s 12ms/step - loss: 0.0114 - accuracy: 0.9964 - val_loss: 0.6089 - val_accuracy: 0.9085


In [37]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [38]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [39]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3117,  302],
       [ 250, 2366]])

In [40]:
print(accuracy_score(y_test,y_pred))

0.9085335542667772
